In [1]:
# %%timeit
import os
sc_fermi_exec = '/Users/bjm42/source/sc-fermi/upload-sourceforge/frozen-sc-fermi'
print(os.popen(sc_fermi_exec).read())

**************************************************************
  (FROZEN)

   SSSS    CCCC      FFFFFF  EEEEEE   RRRR   MM     MM  IIIII
  SS   S  CC   C     FF      EE      RR   R  MMM   MMM    I
  SS      CC         FF      EE      RR  R   M MM MM M    I
   SSSS   CC     --- FFFFFF  EEEEEE  RRRR    M  MMM  M    I
      SS  CC         FF      EE      R   R   M   M   M    I
  S   SS  CC   C     FF      EE      R   RR  M   M   M    I
   SSSS    CCCC      FF      EEEEEE  R   RR  M   M   M  IIIII


Energies in eV, temperature in Kelvin, DOS in states/unit cell

------
j.buckeridge@ucl.ac.uk 2018
**************************************************************

unitcell.dat found...
(Should be cell for which DOS was determined!)
 
Volume of cell:    1056.622886 A^3
 
Found non-spin polarised system...
 
Number of electrons in system:     544.000000
         Energy gap of system:       5.904000 eV
                 Temperature :     500.000000 K
     Number of defect species:    14
 
Defects f

In [ ]:
read_input_data?

In [2]:
# %%timeit

from pymatgen.io.vasp import Poscar
from py_sc_fermi.inputs import read_input_data, read_dos_data, read_unitcell_data
from py_sc_fermi.defect_system import DefectSystem
from py_sc_fermi.dos import DOS

# volume = Poscar.from_file('POSCAR').structure.volume
inputs = read_input_data(filename='./input-fermi-frozen.dat', frozen=True)
volume = read_unitcell_data(filename='./unitcell.dat')
dos = read_dos_data('./totdos.dat', nelect=inputs['nelect'], egap=inputs['egap'])
ds = DefectSystem(defect_species=inputs['defect_species'],
                  dos=dos, 
                  temperature=inputs['temperature'], 
                  volume=volume)
print(ds)
ds.get_sc_fermi(verbose=False)


Found spin polarised system...
Number of electrons in system: 544
Energy gap of system: 5.904 eV
Temperature: 500.0 K
Number of defect species: 14
Volume of cell: 1056.622886347127 A^3
Reading spin-polarised DOS
Integration of DOS up to Fermi level: 542.8624039950018
Renormalised integrated DOS        : 544.0
DefectSystem
  nelect: 544 e
  egap:   5.904 eV
  volume: 1056.622886347127 A^3
  temperature: 500.0 K

Contains defect species:

v_O, nsites=3
  q=+2.0, e=-3.9226860242378474, deg=1
  q=+1.0, e=-0.37983636105949087, deg=2
  q=+0.0, e=4.0309776500000645, deg=1

v_Li, nsites=3
  q=+0.0, e=3.17135738000001, deg=2
  q=-1.0, e=3.659671988940586, deg=1

Li_i, nsites=1
  q=+0.0, e=2.9462461100000685, deg=2
  q=+1.0, e=-2.6395306397551033, deg=1

O_i, nsites=1
  q=+0.0, e=4.208925799999978, deg=1
  q=-1.0, e=9.915873548940516, deg=1

v_La, nsites=2
  q=-3.0, e=12.270599598259025, deg=1

v_Zr, nsites=1
  q=-4.0, e=17.25056016913296, deg=1

Zr_i, nsites=1
  q=+0.0, e=11.345774575000064, de

/Users/bjm42/source/py-sc-fermi/py_sc_fermi/dos.py:94: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((e_fermi - edos)/kT))
/Users/bjm42/source/py-sc-fermi/py_sc_fermi/dos.py:97: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((edos - e_fermi)/kT))
/Users/bjm42/source/py-sc-fermi/py_sc_fermi/defect_charge_state.py:79: RuntimeWarning: overflow encountered in exp
  return self.degeneracy * np.exp(expfac)


3.12593351705938

In [3]:
# %%timeit
ds.get_sc_fermi(verbose=False, emin=0.0)

3.12593351705938

In [4]:
ds.get_sc_fermi_new(verbose=False, emin=0.0)

{'e_fermi': 3.1259335170592353,
 'n_iter': 47,
 'bracket': [3.1259335170592353, 3.1259335170594644]}

In [5]:
ds.report()

SC Fermi level :      3.12593351705938  (eV)

Concentrations:
n (electrons)  : 5.529780157673606e-08 cm^-3
p (holes)      : 7.939617197839504e-11 cm^-3
v_O            : 0.009466097927346736 cm^-3
v_Li           : 1.1840234568974868e+16 cm^-3
Li_i           : 1.1840234568861858e+16 cm^-3
O_i            : 3.564700657261788e-22 cm^-3
v_La           : 1.3152607736924047e-08 cm^-3
v_Zr           : 1.3496861406397588e-27 cm^-3
Zr_i           : 9.898450015237556e-15 cm^-3
Zr_Li_tet      : 1.4397722414040288e-11 cm^-3
Zr_Li          : 27930.445685600494 cm^-3
Zr_La          : 109522.4679455258 cm^-3
La_Zr          : 14.909931188530404 cm^-3
Li_La          : 32197.643290134452 cm^-3
Li_Zr          : 518.2032337354948 cm^-3
La_Li          : 3.2570002449193433e-07 cm^-3

Breakdown of concentrations for each defect charge state:
---------------------------------------------------------
v_O        : Charge Concentration(cm^-3) Total
           :  2.0  9.464910e-03          99.99 
           :  1.0 

In [ ]:
c_v_Li = ds.defect_species_by_name('v_Li').get_concentration(e_fermi=3.378797377976006, temperature=ds.temperature)

In [ ]:
c_Li_i = ds.defect_species_by_name('Li_i').get_concentration(e_fermi=3.378797377976006, temperature=ds.temperature)

In [ ]:
(c_v_Li - c_Li_i)

In [ ]:
ds.get_sc_fermi()

In [ ]:
ds.get_constrained_sc_fermi(constraint={'v_Li': +1, 'Li_i': -1}, total=-0.3)